### **Load Data**

In [ ]:
import pandas as pd

# Load data set

def load_data(file_path):
    df = pd.read_csv(file_path)
    
    return df

file_path = '../data/flight_df.csv'
df = load_data(file_path)

df.head()

: 

### **Model Validation**

In [ ]:
from sklearn.model_selection import train_test_split

# Define target variable
y =df['satisfied']
X = df.drop(columns='satisfied')

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Check for class imbalance
y.value_counts(normalize=True)

### **Preprocessing**

In [ ]:
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

# Select numeric and categorical columns
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_include='object')

# Instantiate transformer
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore')
imputer = KNNImputer(n_neighbors=2, weights='uniform') # Use KNN algorithm to impute missing values to preserve data integrity

# preprocessing pipeline
def preprocessing_pipeline():
    
    num_pipe = Pipeline([
        ('scaler', scaler),
        ('imputer', imputer)
    ])
    
    cat_pipe = Pipeline([
        ('encoder', encoder)
    ])
    
    preprocessor = ColumnTransformer([
        ('numeric', num_pipe, num_cols),
        ('categorical', cat_pipe, cat_cols)
    ])
    
    return preprocessor

# Create preprocessor object
preprocessor = preprocessing_pipeline()

In [ ]:
from imblearn.over_sampling import SMOTE # Synthetic Minority Oversampling TEchnique
from imblearn. pipeline import Pipeline as ImbalancePipeline

# Create sampler pipeline
def sampler_pipeline(sampler):
    
    return ImbalancePipeline([
        ('sampler', sampler)
    ])
    
# Create sampler object
sampler = sampler_pipeline(SMOTE(random_state=42))

# preprocess and rebalance the dataset
def preprocess_and_rebalance(preprocessor, X_train, X_test, y_train):
    
    # transform training data to the fitted transformer
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    # Rebalance training load_data
    X_train_balanced, y_train_balanced = sampler.fit_resample(X_train_transformed, y_train)
    
    return X_train_balanced, X_test_transformed, y_train_balanced

# Preprocess and Rebalance the dataset

X_train_balanced, X_test_transformed, y_train_balanced = preprocess_and_rebalance(preprocessor, X_train, X_test, y_train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Dark mode
plt.style.use('dark_background')

# Visualize data before SMOTE
def plot_distribution(y, title):
    plt.figure(figsize=(5, 3))
    sns.countplot(
        x=y, # y is the target variable 'satisfied'
        palette = 'pastel',
        hue=y
    )
    
    plt.title(title)
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.show()
    
# Visualize data before SMOTE
plot_distribution(y, title='Class Distribution Before SMOTE')

# Visualize data after SMOTE
plot_distribution(y_train_balanced, title='Class Distribution After SMOTE')

In [ ]:
# Convert preprocessed data back to a dataframe
X_train_balanced_df = pd.DataFrame(X_train_balanced)
X_test_transformed_df = pd.DataFrame(X_test_transformed)

### **Machine Learning**

In [ ]:
from lazypredict.Supervised import LazyClassifier
# Instantiate the model
lazy_clf = LazyClassifier(verbose=0, ignore_warnings=False, custom_metric=None)
# Make predictions
models, predictions = lazy_clf.fit(X_train_balanced_df, X_test_transformed_df, y_train_balanced, y_test)
# Return models
models

In [ ]:
# Return top 5 models
top_5_models = models.sort_values('F1 Score', ascending=False).head()
top_5_models

### **Evaluation**

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Function to evaluate model performance
def evaluate_model(model, X_train, y_train, X_test, y_test):
    
    # Make Predictions
    train_preds = np.rint(model.predict(X_train))
    test_preds = np.rint(model.predict(X_test))
    
    # Classification report
    train_report = classification_report(y_train, train_preds)
    test_report = classification_report(y_test, test_preds)
    
    # Confusion matrix
    train_cn = confusion_matrix(y_train, train_preds)
    test_cn = confusion_matrix(y_test, test_preds)
    
    # Define labels
    labels = ['Not Satisfied', 'Satisfied']
    
    # Plot confusion matrices side-by-side Training
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # First axis
    axes[0].test(0.01, 0.05, str(train_report), fontproperties='monospace')
    axes[0].axis('off')
    
    # Second axis
    disp_train = ConfusionMatrixDisplay(confusion_matrix=train_cn, display_labels=labels)
    disp_train.plot(ax=axes[1], cmap='Blues')
    axes[1].set_title('Training Confusion Matrix')
    
    # Plot confusion matrices side-by-side Testing
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # First axis
    axes[0].test(0.01, 0.05, str(test_report), fontproperties='monospace')
    axes[0].axis('off')
    
    # Second axis
    disp_test = ConfusionMatrixDisplay(confusion_matrix=test_cn, display_labels=labels)
    disp_test.plot(ax=axes[1], cmap='Blues')
    axes[1].set_title('Testing Confusion Matrix')
    
    plt.show()
    
    return train_report, test_report

In [ ]:
# ML pipeline
def train_and_predict(X_train, y_train, X_test, preprocessor, model):
    
    # Complete pipeline and model
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Fit model
    model_pipeline.fit(X_train, y_train)
    
    # Save predictions
    train_preds = model_pipeline.predict(X_train)
    test_preds = model_pipeline.predict(X_test)
    
    return train_preds, test_preds

In [ ]:
import lightgbm as lgb

# Instantiate the model
lgb_model = lgb.LGBMClassifier(verbose=-1, random_state=42)

# Create and save predictions
train_preds, test_preds = train_and_predict(X_train_balanced_df, y_train_balanced, X_test_transformed_df, preprocessor, lgb_model)

# Return train and test reports
train_report, test_report = evaluate_model(lgb_model, X_train_balanced_df, y_train_balanced, X_test_transformed_df, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate the model
dec_tree = DecisionTreeClassifier()

# Create and save predictions
train_preds, test_preds = train_and_predict(X_train_balanced_df, y_train_balanced, X_test_transformed_df, preprocessor, dec_tree)

# Return train and test reports
train_report, test_report = evaluate_model(lgb_model, X_train_balanced_df, y_train_balanced, X_test_transformed_df, y_test)

### **Feature Importanc**